In [56]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
import joblib


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

pd.options.display.float_format = '{:.2f}'.format


In [57]:
#data = pd.read_csv(r"./data/2019-20/player_past_history/merged_data.csv" , parse_dates = ["kickoff_time"]) 
#all_players = pd.read_csv(r"./data/2019-20/all_players.csv" , parse_dates= ["news_added"]) 

data = pd.read_csv(r"./data/2020-21/Merged_data/player_gameweek_history.csv" , parse_dates = ["kickoff_time"]) 
all_players = pd.read_csv(r"./data/2020-21/Merged_data/all_players_merged.csv" , parse_dates= ["news_added"]) 

total_players = len(all_players)
orig_df = data
data = data[data.minutes != 0]



In [58]:
orig_df.shape

(3849, 36)

In [59]:
data.shape

(1866, 36)

In [60]:
data.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,player_type_id,PPG,PP90,team_strength,team_id
15,3,9,19,1,True,2020-09-19 19:00:00,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,3,0.00,0.00,0.00,0.00,55,-23455,49797,1738,25193,2,1.40,1.97,4,1
16,3,23,11,1,False,2020-09-28 19:00:00,3,1,3,90,0,0,0,3,0,0,0,0,0,0,0,11,20.20,3.20,0.00,2.30,55,711,61983,7407,6696,2,1.40,1.97,4,1
17,3,29,15,2,True,2020-10-04 13:00:00,2,1,4,90,0,0,0,1,0,0,0,0,0,0,0,13,23.00,0.80,0.00,2.40,55,1264,69592,6180,4916,2,1.40,1.97,4,1
18,3,44,12,2,False,2020-10-17 16:30:00,1,0,5,90,0,0,0,1,0,0,0,0,0,0,0,12,14.60,3.30,19.00,3.70,55,-1132,73952,5063,6195,2,1.40,1.97,4,1
19,3,49,9,1,True,2020-10-25 19:15:00,0,1,6,48,0,0,0,0,0,0,0,0,0,0,0,12,8.80,2.50,2.00,1.30,55,2396,79892,6821,4425,2,1.40,1.97,4,1


In [61]:
data.columns

Index(['player_id', 'fixture_id', 'opponent_team_id', 'total_points',
       'was_home', 'kickoff_time', 'team_h_score', 'team_a_score', 'round',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'value', 'transfers_balance', 'selected',
       'transfers_in', 'transfers_out', 'player_type_id', 'PPG', 'PP90',
       'team_strength', 'team_id'],
      dtype='object')

In [62]:
data = data[[
    'player_id' , 
    'opponent_team_id' , 
    'total_points' , 
    'was_home' , 
    'minutes' , 
    'player_type_id' , 
    #'PPG' , 
    #'PP90' , 
    'team_strength' , 
    'team_id'
    ]]

In [63]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1866 entries, 15 to 3841
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   player_id         1866 non-null   int64
 1   opponent_team_id  1866 non-null   int64
 2   total_points      1866 non-null   int64
 3   was_home          1866 non-null   bool 
 4   minutes           1866 non-null   int64
 5   player_type_id    1866 non-null   int64
 6   team_strength     1866 non-null   int64
 7   team_id           1866 non-null   int64
dtypes: bool(1), int64(7)
memory usage: 118.4 KB


In [64]:
#data['team_id'] = data['team_id'].astype(str)

In [65]:
corr_matrix = data.corr()
corr_matrix['total_points'].sort_values(ascending=False)

total_points        1.00
minutes             0.28
team_strength       0.10
player_type_id      0.04
player_id           0.00
opponent_team_id    0.00
team_id            -0.01
was_home           -0.07
Name: total_points, dtype: float64

In [66]:
# Replace categorical data with one-hot encoded data
features_data = pd.get_dummies(data, columns=['team_id' , 'opponent_team_id' , 'team_strength' , 'player_type_id' , 'was_home'])
#features_df = pd.get_dummies(df)

features_data = features_data.drop('total_points', axis=1)

list_features = list(features_data.columns.values)
print(list_features)

['player_id', 'minutes', 'team_id_1', 'team_id_2', 'team_id_3', 'team_id_4', 'team_id_5', 'team_id_6', 'team_id_7', 'team_id_8', 'team_id_9', 'team_id_10', 'team_id_11', 'team_id_12', 'team_id_13', 'team_id_14', 'team_id_15', 'team_id_16', 'team_id_17', 'team_id_18', 'team_id_19', 'team_id_20', 'opponent_team_id_1', 'opponent_team_id_2', 'opponent_team_id_3', 'opponent_team_id_4', 'opponent_team_id_5', 'opponent_team_id_6', 'opponent_team_id_7', 'opponent_team_id_8', 'opponent_team_id_9', 'opponent_team_id_10', 'opponent_team_id_11', 'opponent_team_id_12', 'opponent_team_id_13', 'opponent_team_id_14', 'opponent_team_id_15', 'opponent_team_id_16', 'opponent_team_id_17', 'opponent_team_id_18', 'opponent_team_id_19', 'opponent_team_id_20', 'team_strength_2', 'team_strength_3', 'team_strength_4', 'team_strength_5', 'player_type_id_1', 'player_type_id_2', 'player_type_id_3', 'player_type_id_4', 'was_home_False', 'was_home_True']


In [67]:
for i in list(data):
    print(i, data[i].values[0],type(data[i].values[0]))

# Create the X and y arrays
X = features_data.values
y = data['total_points'].values

# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


player_id 3 <class 'numpy.int64'>
opponent_team_id 19 <class 'numpy.int64'>
total_points 1 <class 'numpy.int64'>
was_home True <class 'numpy.bool_'>
minutes 1 <class 'numpy.int64'>
player_type_id 2 <class 'numpy.int64'>
team_strength 4 <class 'numpy.int64'>
team_id 1 <class 'numpy.int64'>


In [68]:
# Create the model
model = ensemble.GradientBoostingRegressor()

In [69]:
#Parameters we want to try
param_grid = {
    'n_estimators': [500, 1000, 3000],
    'max_depth': [4, 6],
    'min_samples_leaf': [3, 5, 9, 17],
    'learning_rate': [0.1, 0.05, 0.02, 0.01],
    'max_features': [1.0, 0.3, 0.1],
    'loss': ['ls', 'lad', 'huber']
    }

## THIS MODULE TAKES A LONG TIME TO RUN

In [70]:
#Define the grid search we want to run. Run it with four cpus in parallel.
#gs_cv = GridSearchCV(model, param_grid, n_jobs=4)

#Run the grid search - on only the training data!
#gs_cv.fit(X_train, y_train)

#Print the parameters that gave us the best result!
#print(gs_cv.best_params_)

# After running a .....long..... time, the output will be something like
# {'loss': 'huber', 'learning_rate': 0.1, 'min_samples_leaf': 9, 'n_estimators': 3000, 'max_features': 0.1, 'max_depth': 6}
# That is the combination that worked best.

### Fit regression model


In [71]:
model = ensemble.GradientBoostingRegressor(
    n_estimators=3000,
    learning_rate=0.1,
    max_depth=4,
    min_samples_leaf=3,
    max_features=1.0,
    loss='huber',
    random_state=0
)

model.fit(X_train, y_train)

GradientBoostingRegressor(loss='huber', max_depth=4, max_features=1.0,
                          min_samples_leaf=3, n_estimators=3000,
                          random_state=0)

### Save the trained model to a file so we can use it in other programs


In [72]:
joblib.dump(model, 'Trained_PlayerAnalysis.pkl')

['Trained_PlayerAnalysis.pkl']

### Find the error rate on the training set


In [73]:
mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

Training Set Mean Absolute Error: 0.4512


### Find the error rate on the test set


In [74]:
mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)

Test Set Mean Absolute Error: 2.2811


In [75]:
# Load the trained model created with train_model.py
model = joblib.load('Trained_PlayerAnalysis.pkl')

# Create a numpy array based on the model's feature importances
importance = model.feature_importances_

# Sort the feature labels based on the feature importance rankings from the model
feature_indexes_by_importance = importance.argsort()


In [76]:
### Print each feature label, from most important to least important (reverse order)


In [77]:
for index in feature_indexes_by_importance:
    print("{} - {:.2f}%".format(list_features[index], (importance[index] * 100.0)))

team_id_12 - 0.08%
team_id_8 - 0.13%
team_id_3 - 0.15%
team_id_17 - 0.16%
team_id_18 - 0.16%
team_id_19 - 0.16%
team_id_1 - 0.21%
team_id_4 - 0.23%
team_id_11 - 0.24%
team_id_7 - 0.39%
opponent_team_id_3 - 0.40%
team_id_16 - 0.41%
team_id_13 - 0.52%
team_id_14 - 0.61%
team_id_6 - 0.61%
team_id_9 - 0.62%
team_id_15 - 0.63%
opponent_team_id_12 - 0.65%
team_strength_5 - 0.68%
opponent_team_id_2 - 0.68%
team_strength_4 - 0.69%
team_id_5 - 0.79%
opponent_team_id_5 - 0.82%
opponent_team_id_16 - 0.83%
opponent_team_id_7 - 0.86%
team_strength_3 - 0.91%
opponent_team_id_1 - 0.95%
opponent_team_id_17 - 0.96%
team_id_2 - 0.97%
team_id_20 - 0.98%
player_type_id_1 - 1.00%
opponent_team_id_14 - 1.06%
team_id_10 - 1.08%
opponent_team_id_10 - 1.14%
opponent_team_id_11 - 1.18%
opponent_team_id_13 - 1.19%
opponent_team_id_9 - 1.32%
team_strength_2 - 1.34%
opponent_team_id_20 - 1.39%
opponent_team_id_6 - 1.39%
player_type_id_4 - 1.50%
opponent_team_id_8 - 1.53%
opponent_team_id_15 - 1.62%
opponent_team_i